# Positive Predictive Value (PPV) Analysis: SPECT/CT Scan

### Objective

Calculate the **PPV of localized SPECT/CT scans** by comparing imaging localization to surgical findings.

### Data Source

- `side_concordance_results.xlsx` (pre-operative imaging and surgical outcomes)

### Outputs

- **Positive Predictive Value (PPV) for localized SPECT/CT scans:** **92.1%**

---

In [7]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/side_concordance_results.xlsx')

In [8]:
def extract_side(location):
    if pd.isnull(location):
        return 'No Scan'
    
    if 'Left' in location:
        return 'Left'
    elif 'Right' in location:
        return 'Right'
    else:
        return 'Unknown'
    
preop_imaging_df['Surgical Side'] = preop_imaging_df['Surgical_Findings'].apply(extract_side)


In [9]:
# Define modality
modality = 'SPECT/CT'

# Apply extract_side function if not already done to get imaging sides
preop_imaging_df['SPECT/CT Imaging Side'] = preop_imaging_df[modality].apply(extract_side)

# Filter to scans that localized anywhere (Left, Right)
localized_spect = preop_imaging_df[preop_imaging_df['SPECT/CT Imaging Side'].isin(['Left', 'Right'])].copy()

# Count total localized scans
total_localized_spect = localized_spect.shape[0]
total_localized_spect

177

In [10]:
def check_concordance(row):
    imaging = row['SPECT/CT']
    surgical = row['Surgical_Findings']
    
    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'
    imaging_glands = [g.strip() for g in imaging.split(',')]
    surgical_glands = [g.strip() for g in surgical.split(',')]
    
    if any(sg in imaging_glands for sg in surgical_glands):
        return 'Yes (Exact)'
    
    imaging_sides = []
    for ig in imaging_glands:
        if 'Left' in ig:
            imaging_sides.append('Left')
        if 'Right' in ig:
            imaging_sides.append('Right')
    
    surgical_sides = []
    for sg in surgical_glands:
        if 'Left' in sg:
            surgical_sides.append('Left')
        if 'Right' in sg:
            surgical_sides.append('Right')
    
    if any(side in surgical_sides for side in imaging_sides):
        return 'Yes (Side)'
    
    return 'Incorrect Localization'


In [11]:
localized_spect['Calculated Concordance'] = localized_spect.apply(check_concordance, axis=1)

false_positives = (localized_spect['Calculated Concordance'] == 'Incorrect Localization').sum()
true_positive = (
    (localized_spect['Calculated Concordance'] == 'Yes (Exact)') |
    (localized_spect['Calculated Concordance'] == 'Yes (Side)')
).sum()

ppv = true_positive / (true_positive + false_positives)

print(f"Positive Predictive Value (PPV) for localized SPECT/CT scans: {ppv:.2%}")

Positive Predictive Value (PPV) for localized SPECT/CT scans: 92.09%


In [12]:
false_positives

14